In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pickle
file = open('dataset.pkl','rb')
DATA = pickle.load(file)
pheno = DATA['pheno']
X_gpa = DATA['X_gpa']
X_snps = DATA['X_snps']
X_genexp = DATA['X_genexp']


In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import recall_score
import numpy as np
import pandas as pd


def refine_and_evaluate_with_mlp(pheno, X_gpa, X_snps, X_genexp, param_grid):
    performances = []
    colonnes_importance = []  #  colonnes triées par importance
    hyperparams_records = []  #  hyperparamètres

    # Pour chaque antibiotique
    for antibiotique in pheno.columns[1:]:
        print(f"Traitement de l'antibiotique : {antibiotique}")

        # Extraire la cible (y) et les indices valides
        y = pheno[antibiotique].to_numpy()  
        valid_indices = ~np.isnan(y)  
        y = y[valid_indices]  

        X_gpa_filtered = X_gpa[valid_indices]  
        X_snps_filtered = X_snps[valid_indices]  
        X_genexp_filtered = X_genexp[valid_indices]  

        # Diviser les données en train/test avec stratification
        X_gpa_train, X_gpa_test, X_snps_train, X_snps_test, X_genexp_train, X_genexp_test, y_train, y_test = train_test_split(
            X_gpa_filtered, X_snps_filtered, X_genexp_filtered, y, 
            test_size=0.2, random_state=42, stratify=y
        )

        scaler_genexp = StandardScaler()
        X_genexp_train = scaler_genexp.fit_transform(X_genexp_train)  # Fit + transform sur l'entraînement
        X_genexp_test = scaler_genexp.transform(X_genexp_test)        # Transform uniquement sur le test

        X_train = np.hstack([X_gpa_train, X_snps_train, X_genexp_train])
        X_test = np.hstack([X_gpa_test, X_snps_test, X_genexp_test])


        grid_search = GridSearchCV(MLPClassifier(random_state=42), param_grid, cv=5, scoring='recall', verbose=1)
        grid_search.fit(X_train, y_train)

        # Meilleur modèle
        meilleur_modele = grid_search.best_estimator_

        # Prédire sur l'ensemble de test
        y_pred = meilleur_modele.predict(X_test)

        # Calculer le macro recall
        rappel_macro = recall_score(y_test, y_pred, average='macro')
        print(f"Rappel macro pour {antibiotique} avec MLP: {rappel_macro:.4f}")

        # Calculer l'importance des colonnes (via les poids)
        importances = np.abs(meilleur_modele.coefs_[0]).mean(axis=1)

        # Associer chaque colonne à son importance
        colonne_labels = (
            [f"GPA_{i}" for i in range(X_gpa_filtered.shape[1])] +
            [f"SNP_{i}" for i in range(X_snps_filtered.shape[1])] +
            [f"GenExp_{i}" for i in range(X_genexp_filtered.shape[1])]
        )
        colonnes_importance_antibiotique = sorted(
            zip(colonne_labels, importances), key=lambda x: x[1]
        )  # Trier par importance croissante

        colonnes_importance += [
            {'Antibiotique': antibiotique, 'Colonne': col, 'Importance': imp}
            for col, imp in colonnes_importance_antibiotique
        ]

        # Stocker les performances
        performances.append({
            'Antibiotique': antibiotique,
            'Rappel_macro': rappel_macro
        })

        # Stocker les meilleurs hyperparamètres 
        hyperparams_records.append({
            'Antibiotique': antibiotique,
            'Meilleur_paramètre': grid_search.best_params_,
            'Valeurs_des_hyperparamètres': param_grid
        })

    performances_df = pd.DataFrame(performances)

    # Sauvegarder fichier CSV
    performances_df.to_csv('result_mlp_performances.csv', index=False, sep=';')

    colonnes_importance_df = pd.DataFrame(colonnes_importance)

    colonnes_importance_df.to_csv('result_mlp_sorted_columns.csv', index=False, sep=';')

    hyperparams_df = pd.DataFrame(hyperparams_records)

    hyperparams_df.to_csv('param_model.csv', index=False, sep=';')

    return performances_df, colonnes_importance_df, hyperparams_df



param_grid = {
    'hidden_layer_sizes': [(32,), (64,)],  # Taille des couches cachées
    'activation': ['relu', 'logistic'],  # Fonction d'activation
    'alpha': [0.0001, 0.001],  # Régularisation L2
    'learning_rate_init': [0.001]  # Taux d'apprentissage initial
}

performances_df, colonnes_importance_df, hyperparams_df = refine_and_evaluate_with_mlp(pheno, X_gpa, X_snps, X_genexp, param_grid)

print("\nPerformances MLP pour chaque antibiotique:")
print(performances_df)

print("\nColonnes triées par importance pour chaque antibiotique:")
print(colonnes_importance_df)

print("\nHyperparamètres pour chaque antibiotique:")
print(hyperparams_df)


Traitement de l'antibiotique : Tobramycin
Fitting 3 folds for each of 8 candidates, totalling 24 fits


c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cbe

Rappel macro pour Tobramycin avec MLP: 0.8977
Traitement de l'antibiotique : Ceftazidim
Fitting 3 folds for each of 8 candidates, totalling 24 fits


c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cbe

Rappel macro pour Ceftazidim avec MLP: 0.5379
Traitement de l'antibiotique : Ciprofloxacin
Fitting 3 folds for each of 8 candidates, totalling 24 fits


...

Rappel macro pour Ciprofloxacin avec MLP: 0.8156
Traitement de l'antibiotique : Meropenem
Fitting 3 folds for each of 8 candidates, totalling 24 fits


c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cbe

Rappel macro pour Meropenem avec MLP: 0.5000
Traitement de l'antibiotique : Colistin
Fitting 3 folds for each of 8 candidates, totalling 24 fits


c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cberger\anaconda3\envs\appstat\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\cbe

Rappel macro pour Colistin avec MLP: 0.5218

Performances MLP pour chaque antibiotique:
    Antibiotique  Rappel_macro
0     Tobramycin      0.897665
1     Ceftazidim      0.537879
2  Ciprofloxacin      0.815625
3      Meropenem      0.500000
4       Colistin      0.521836

Colonnes triées par importance pour chaque antibiotique:
       Antibiotique      Colonne  Importance
0        Tobramycin     SNP_5690    0.000915
1        Tobramycin    SNP_19727    0.000926
2        Tobramycin     GPA_5727    0.000936
3        Tobramycin     SNP_8655    0.000953
4        Tobramycin     SNP_5689    0.000955
...             ...          ...         ...
471330     Colistin  GenExp_5671    0.010570
471331     Colistin   GenExp_763    0.010689
471332     Colistin   GenExp_392    0.010804
471333     Colistin  GenExp_5166    0.011017
471334     Colistin  GenExp_1482    0.011143

[471335 rows x 3 columns]

Hyperparamètres pour chaque antibiotique:
    Antibiotique                                 Meilleur_

In [2]:
import pandas as pd

colonnes_importance_df = pd.read_csv('result_mlp_sorted_columns.csv', sep=';')
performances_df = pd.read_csv('performances_selection.csv', sep=';')

colonnes_importance_df_sorted = colonnes_importance_df.sort_values(by='Importance', ascending=False)

# Garder uniquement les top_k colonnes pour chaque antibiotique
top_k = 1000
top_columns = (
    colonnes_importance_df_sorted.groupby("Antibiotique")
    .head(top_k)
)

# Organiser les colonnes par type (GPA, SNP, GenExp)
top_columns["Type"] = top_columns["Colonne"].str.split("_").str[0]

# Regrouper par antibiotique pour chaque type de colonne
gpa_columns = top_columns[top_columns["Type"] == "GPA"].groupby("Antibiotique")["Colonne"].apply(list)
snps_columns = top_columns[top_columns["Type"] == "SNP"].groupby("Antibiotique")["Colonne"].apply(list)
genexp_columns = top_columns[top_columns["Type"] == "GenExp"].groupby("Antibiotique")["Colonne"].apply(list)

# Extraire les indices pour chaque type de colonne (GPA, SNPs, GenExp)
gpa_indices = top_columns[top_columns["Type"] == "GPA"].groupby("Antibiotique")["Colonne"].apply(lambda x: [int(col.split('_')[1]) for col in x]).fillna("[]")
snps_indices = top_columns[top_columns["Type"] == "SNP"].groupby("Antibiotique")["Colonne"].apply(lambda x: [int(col.split('_')[1]) for col in x]).fillna("[]")
genexp_indices = top_columns[top_columns["Type"] == "GenExp"].groupby("Antibiotique")["Colonne"].apply(lambda x: [int(col.split('_')[1]) for col in x]).fillna("[]")

# Ajouter ces informations au DataFrame des performances
performances_df["Colonnes_GPA_Indices"] = performances_df["Antibiotique"].map(gpa_indices).fillna("[]")
performances_df["Colonnes_SNPs_Indices"] = performances_df["Antibiotique"].map(snps_indices).fillna("[]")
performances_df["Colonnes_Expression_Genetique_Indices"] = performances_df["Antibiotique"].map(genexp_indices).fillna("[]")

# Ajouter l'importance pour chaque colonne
top_columns["Importance"] = top_columns["Importance"].round(4)  # Arrondir l'importance pour plus de lisibilité
importance_dict = top_columns.groupby("Antibiotique").apply(lambda group: dict(zip(group["Colonne"], group["Importance"]))).to_dict()
performances_df["Importance"] = performances_df["Antibiotique"].map(importance_dict)

performances_df.rename(columns={"Rappel_macro": "Recall"}, inplace=True)

performances_df[["Antibiotique", "Recall", "Importance", "Colonnes_GPA_Indices", "Colonnes_SNPs_Indices", "Colonnes_Expression_Genetique_Indices"]].to_csv("performances_with_columns_and_importance.csv", sep=";", index=False)

print("Le fichier CSV 'performances_with_columns_and_importance.csv' a été généré.")


Le fichier CSV 'performances_with_columns_and_importance.csv' a été généré.


C:\Users\cberger\AppData\Local\Temp\ipykernel_13580\713478239.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_columns["Type"] = top_columns["Colonne"].str.split("_").str[0]
C:\Users\cberger\AppData\Local\Temp\ipykernel_13580\713478239.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_columns["Importance"] = top_columns["Importance"].round(4)  # Arrondir l'importance pour plus de lisibilité


In [11]:
import pandas as pd
import matplotlib.pyplot as plt



def modify_dataframe(file_path, output_path):
    # Charger le DataFrame
    df = pd.read_csv(file_path, sep=';')  

    # Supprimer la colonne 'Importance'
    if 'Importance' in df.columns:
        df = df.drop(columns=['Importance'])

    # Renommer les colonnes
    df = df.rename(columns={
        'Colonnes_GPA_Indices': 'Colonnes_GPA',
        'Colonnes_SNPs_Indices': 'Colonnes_SNPs',
        'Colonnes_Expression_Genetique_Indices': 'Colonnes_Expression_Genetique'
    })

    # Sauvegarder le DataFrame modifié
    df.to_csv(output_path, sep=';', index=False)
    print(f"Fichier traité et sauvegardé dans : {output_path}")

# Exemple d'utilisation
modify_dataframe('performances_with_columns_and_importance.csv', 'output.csv')

Fichier traité et sauvegardé dans : output.csv
